# 총평점, a_isbn 수집
수집된 item_id을 통해 총 평점(total_rating), a_isbn 수집 
- 사용파일: A_aladin_list_6_final_utf8.csv(알라딘 베트스셀러>역대베스트 2015, 2019~2023 총 6개년 엑셀 파일을 다운로드 받아 생성)
- 생성파일: information_etc_utf8.csv

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import json
from selenium import webdriver
import time
import csv
import pandas as pd

In [2]:
# CSV 파일 경로
file_path = 'A_aladin_list_6_final_utf8.csv'

# pandas를 사용하여 CSV 파일 읽기
df = pd.read_csv(file_path, encoding='utf-8')

# 'item_id' 열의 값들을 중복 없이 리스트로 변환
item_id_list = df['item_id'].unique().tolist()

In [3]:
len(item_id_list)

1025

In [4]:
information_etc_list = []
information_etc_html_list = []

In [5]:
wd = webdriver.Chrome()
count = 1
for item_id in item_id_list:
    #초기회 방법 수정
    a_isbn = categories = page = None
    try:
        #알라딘상세페이지
        aladin_url =  f'https://www.aladin.co.kr/shop/wproduct.aspx?ItemId={item_id}'
        wd.get(aladin_url)
        # 브라우져가 로딩이 완료될때까지 기다려야한다.
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 3초동안 기다림
        html = wd.page_source
        soup = BeautifulSoup(html,'html.parser')
        # print(soup.prettify())
        
        #a_isbn
        aladin_isbn_soup = soup.select_one('#wa_product_top1_wa_Top_Ranking_pnlRanking > div.info_list.Ere_fs15.Ere_ht18 > a:nth-child(5)')
        if aladin_isbn_soup:
            aladin_isbn_href = aladin_isbn_soup.get('href')
            # '#'과 '_' 사이의 값을 추출
            a_isbn = aladin_isbn_href.split('#')[-1].split('_')[0]
        else:
            a_isbn = None

        #전체평점
        total_rating_soup = soup.select_one('#wa_product_top1_wa_Top_Ranking_pnlRanking > div.info_list.Ere_fs15.Ere_ht18 > a.Ere_sub_pink.Ere_fs16.Ere_str')
        if total_rating_soup:
            total_rating = total_rating_soup.text.strip()
        else:
            total_rating = None
        
        information_etc_dict = {
            'item_id': item_id,
            'a_isbn': a_isbn,
            'total_rating': total_rating,
        }
        information_etc_list.append(information_etc_dict)
        print(count, time.strftime('%Y.%m.%d - %H:%M:%S'), item_id, a_isbn, total_rating)
        count += 1
    except Exception as e:
        information_etc_dict = {
            'item_id': item_id,
            'a_isbn': a_isbn,
            'total_rating': total_rating,
        }
        information_etc_list.append(information_etc_dict)
        continue


1 2024.03.19 - 11:20:08 176021392 8965422787 9.1


In [43]:
#누락된 item_id있는지 확인
missing_item_ids = set(item_id_list) - set([info['item_id'] for info in information_etc_list])

if missing_item_ids:
    print("Missing item_ids:")
    for item_id in missing_item_ids:
        print(item_id)
else:
    print('누락된 아이디 없음')

Missing item_ids:
272252933
207826954
238612493
6946832
291684381
242925601
58228775
244899880
182313000
999466
68341808
242905138
323084340
275419192
233312327
251080776
210116681
277364809
179220561
247427156
246390870
230965334
317702230
307634266
491611
42770523
199045231
21680244
253833337
302137475
222371972
59420804
288845971
160821395
306405534
62136479
222716064
236708015
191910066
243077303
51552439
282587324
187789500
43532478
235696342
297816279
33345754
8048868
230957285
253133041
276918514
267223294
311329023
151335169
48988418
317300995
261206279
285688072
270815497
238162190
208576782
311984400
259678494
214397215
240816415
259678495
105861410
62288159
56869156
253522213
169566506
51323180
295665964
172998958
296517936
236155192
246808890
313250109
196542784
286859586
288284994
288756034
258343245
11489614
303706448
233423187
259678547
240886101
311005527
291328350
242925920
283685224
179700075
298570092
233320813
62493044
162341238
124830071
204603766
169566585
1969852

In [42]:
#판다스로 중복제거
df = pd.DataFrame(information_etc_list)

# 'item_id'에서 중복된 행만 필터링
duplicated_rows = df[df.duplicated('item_id', keep=False)]

# 결과 출력
if len(duplicated_rows) >= 1:
   print(duplicated_rows.sort_values(by='item_id'))
else:
    print('중복된 행이 없습니다')
# 중복된 행이 있는 경우 삭제
# df.drop_duplicates(subset=['item_id'], keep='first', ignore_index=True)

중복된 행이 없습니다


In [10]:
#저장

fields = list(information_etc_list[0].keys())
# 'encoding' 인자를 'utf-8'로 설정
with open('information_etc_utf8.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(information_etc_list)